### Functional API
The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs. The main idea is that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build graphs of layers.

<p align='center'>
<img src="https://keras.io/img/guides/functional_api/functional_api_40_0.png"/>
</p>

The sequential API allows you to create models layer-by-layer for most problems. It is limited in that it does not allow you to create models that share layers or have multiple inputs or outputs.

The functional API in Keras is an alternate way of creating models that offers a lot more flexibility, including creating more complex models.

> 🎈 With the  Functional **API** we can be able to create the models of the following nature beacuse of it's flexebility.



```
      input                          input               input1   input2                
        |                              |                    \     /                      
      hidden layer(s)               hidden layer(s)        hidden layer(s)             
        |                            /    \                    |                         
      output                    output1  output2              output                     
 Functional & Sequential           (only Functional)       (only Functional)        
```

```
     input
     /  \
   hl    hl
    \   /    
    output
(only Functional) 
```

There a a lot of cases we can use the functional API

In [1]:
import tensorflow as tf
from tensorflow import keras

> Let's consider the following `Sequential` Model.

In [6]:
s_model = keras.Sequential([
    keras.layers.Conv2D(100, 5, activation='tanh', name='first_layer', input_shape=(224,224,3)),
    keras.layers.Conv2D(50, 5, activation='tanh', name='second_layer'),
    keras.layers.Conv2D(32, 5, activation='tanh', name='third_layer'),
    keras.layers.Flatten(name='flatten_layer'),
    keras.layers.Dense(64, activation='relu', name='fifthy_layer'),
    keras.layers.Dense(1, activation='sigmoid', name='output_layer'),
], name='s_model')

s_model.summary()

Model: "s_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_layer (Conv2D)         (None, 220, 220, 100)     7600      
_________________________________________________________________
second_layer (Conv2D)        (None, 216, 216, 50)      125050    
_________________________________________________________________
third_layer (Conv2D)         (None, 212, 212, 32)      40032     
_________________________________________________________________
flatten_layer (Flatten)      (None, 1438208)           0         
_________________________________________________________________
fifthy_layer (Dense)         (None, 64)                92045376  
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 92,218,123
Trainable params: 92,218,123
Non-trainable params: 0
_______________________________________________

> Now let's convert the `s_model` to the functional model `f_model`.

#### Method 1.

In [10]:
input_layer = keras.layers.Input(shape=(224, 224, 3), name="input_layer")
first_layer = keras.layers.Conv2D(100, 5, activation='tanh', name='first_layer')
second_layer = keras.layers.Conv2D(50, 5, activation='tanh', name='second_layer')
third_layer = keras.layers.Conv2D(32, 5, activation='tanh', name='third_layer')
flatten_layer = keras.layers.Flatten(name='flatten_layer')
fifthy_layer = keras.layers.Dense(64, activation='relu', name='fifthy_layer')
output_layer =keras.layers.Dense(1, activation='sigmoid', name='output_layer')

### These layers are callable
x = first_layer(input_layer)
x = second_layer(x)
x = third_layer(x)
x = flatten_layer(x)
x = fifthy_layer(x)
x = output_layer(x)

f1_model = keras.Model(inputs=input_layer, outputs=x, name="f1_model")
f1_model.summary()

Model: "f1_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
first_layer (Conv2D)         (None, 220, 220, 100)     7600      
_________________________________________________________________
second_layer (Conv2D)        (None, 216, 216, 50)      125050    
_________________________________________________________________
third_layer (Conv2D)         (None, 212, 212, 32)      40032     
_________________________________________________________________
flatten_layer (Flatten)      (None, 1438208)           0         
_________________________________________________________________
fifthy_layer (Dense)         (None, 64)                92045376  
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65 

#### Method 2.

In [12]:
input_layer = keras.layers.Input(shape=(224, 224, 3), name="input_layer")

first_layer = keras.layers.Conv2D(100, 5, activation='tanh', name='first_layer')(input_layer)
second_layer = keras.layers.Conv2D(50, 5, activation='tanh', name='second_layer')(first_layer)
third_layer = keras.layers.Conv2D(32, 5, activation='tanh', name='third_layer')(second_layer)
flatten_layer = keras.layers.Flatten(name='flatten_layer')(third_layer)
fifthy_layer = keras.layers.Dense(64, activation='relu', name='fifthy_layer')(flatten_layer)
output_layer =keras.layers.Dense(1, activation='sigmoid', name='output_layer')(fifthy_layer)

f2_model = keras.Model(inputs=input_layer, outputs=output_layer, name="f1_model")
f2_model.summary()

Model: "f1_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
first_layer (Conv2D)         (None, 220, 220, 100)     7600      
_________________________________________________________________
second_layer (Conv2D)        (None, 216, 216, 50)      125050    
_________________________________________________________________
third_layer (Conv2D)         (None, 212, 212, 32)      40032     
_________________________________________________________________
flatten_layer (Flatten)      (None, 1438208)           0         
_________________________________________________________________
fifthy_layer (Dense)         (None, 64)                92045376  
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65 

> The ``method 2`` is one of the common used method in practice but they both yield the same result.

### Convert ``Sequential`` to ``Functional``.
* Create an input layer with a good shape.
* pass the input layer to the first layer of the `Sequentail` model and store in a variable `x`.
* pass the output `x` to all the layers of the model to get the output layer
* finaly create the model with the ``input = input_layer`` and ``output = x`` using `keras.Model()` class.

> 💎 Very important consept when doing `Transfer Learning`.

In [18]:
input_layer = keras.layers.Input(shape=(224, 224, 3))

x = s_model.layers[0](input_layer)

for layer in s_model.layers[1:]:
    x = layer(x)
f3_model = keras.Model(inputs=input_layer, outputs=x, name="f3_model")
f3_model.summary()

Model: "f3_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
first_layer (Conv2D)         (None, 220, 220, 100)     7600      
_________________________________________________________________
second_layer (Conv2D)        (None, 216, 216, 50)      125050    
_________________________________________________________________
third_layer (Conv2D)         (None, 212, 212, 32)      40032     
_________________________________________________________________
flatten_layer (Flatten)      (None, 1438208)           0         
_________________________________________________________________
fifthy_layer (Dense)         (None, 64)                92045376  
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65 

### Convert ``Functional`` to ``Sequential``.
> ✔ This only works if the `functional` model is **linear**.

We need to loop through all the layers of the functional model and add them to our sequential model.

> 💎 Very important consept when doing `Transfer Learning`.

In [13]:
s1_model = keras.Sequential()

for layer in f1_model.layers:
    s1_model.add(layer)
s1_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_layer (Conv2D)         (None, 220, 220, 100)     7600      
_________________________________________________________________
second_layer (Conv2D)        (None, 216, 216, 50)      125050    
_________________________________________________________________
third_layer (Conv2D)         (None, 212, 212, 32)      40032     
_________________________________________________________________
flatten_layer (Flatten)      (None, 1438208)           0         
_________________________________________________________________
fifthy_layer (Dense)         (None, 64)                92045376  
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 92,218,123
Trainable params: 92,218,123
Non-trainable params: 0
____________________________________________

#### Access inputs and outputs for ``model``

In [19]:
f1_model.inputs, f1_model.outputs

([<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_layer')>],
 [<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'output_layer')>])

#### Access input and output for a ``layer``

In [24]:
f1_model.layers[-1].input, f1_model.layers[-1].output

(<KerasTensor: shape=(None, 64) dtype=float32 (created by layer 'fifthy_layer')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'output_layer')>)

> ✔ **Note:** When your Functional model accepts more than `1` input and yield more than `1` output then we pass the input and output as an array. For example:

```
model = keras.Model(inputs=[input_1, input_2,..], outputs=[output_1, output_2, ...])
```
This means when you compile your model each output will have it's loss function. We will see this more in practical examples.